In [1]:
from typing import List, Dict, Set, Any, Optional, Tuple, Literal, Callable
import numpy as np
import torch
from torch import Tensor
import sigkernel
import os
import sys
import tslearn
import tslearn.metrics
import ksig

from kernels.abstract_base import TimeSeriesKernel, StaticKernel
from kernels.static_kernels import LinearKernel, RBFKernel, PolyKernel
from kernels.integral_type import StaticIntegralKernel
from kernels.sig_pde import SigPDEKernel
from kernels.sig_trunc import TruncSigKernel
from kernels.gak import GlobalAlignmentKernel

In [ ]:
# Test how jitting works

test_X = torch.rand(2, 100, 3).cuda()
X = torch.rand(20, 100, 20).cuda()

ker = TruncSigKernel(RBFKernel(), 5, 1, only_last=True, max_batch=100)
out = ker(test_X, test_X)
print(out.shape)

In [ ]:
ker(X,X).shape

In [5]:
#### Test GAK ####
N=20
N2=10
T, d = 70, 2
torch.manual_seed(0)
X = torch.randn(N, T, d, dtype=torch.float64).to("cuda") / d
Y = torch.randn(N2, T, d, dtype=torch.float64).to("cuda") / d
X_np = X.cpu().numpy()
Y_np = Y.cpu().numpy()

sigma = tslearn.metrics.sigma_gak(X_np)
gak = tslearn.metrics.cdist_gak
ksiggak = ksig.kernels.GlobalAlignmentKernel(static_kernel=ksig.static.kernels.RBFKernel(bandwidth=sigma))
mine = GlobalAlignmentKernel(RBFKernel(sigma=1/sigma**2/2), normalize=True)

#out = gak(X, X, sigma=sigma)
out2 = ksiggak(X_np, X_np)
out3 = mine(X, X)
print("out\n\n\n")
#print(out)
print(out2)
print(out3)
print(np.mean(np.abs(out2 - out3.cpu().numpy())))

out



[[1.         0.73627027 0.77307606 0.74218511 0.81982299 0.66653815
  0.69460768 0.67596821 0.7158322  0.66307154 0.66399594 0.70453879
  0.73837668 0.62950863 0.72073267 0.84827092 0.76413235 0.71302505
  0.753775   0.7351122 ]
 [0.73627027 1.         0.78439778 0.79982068 0.70482213 0.7060493
  0.6912417  0.73804002 0.71649658 0.79378332 0.76641393 0.77752324
  0.84567582 0.70935137 0.81459535 0.75696288 0.76499484 0.715232
  0.87216841 0.82387521]
 [0.77307606 0.78439778 1.         0.82731218 0.81520775 0.74850455
  0.76585649 0.85086859 0.82564942 0.74849673 0.83113054 0.78320897
  0.75446186 0.78611742 0.86107949 0.88603931 0.80691759 0.86521207
  0.82862618 0.80563945]
 [0.74218511 0.79982068 0.82731218 1.         0.81433598 0.71615435
  0.7272935  0.77051716 0.76369119 0.74869885 0.77249096 0.79035821
  0.76305924 0.71984038 0.83096153 0.81083461 0.77009325 0.79414197
  0.86296792 0.83241188]
 [0.81982299 0.70482213 0.81520775 0.81433598 1.         0.67816269
  0.63621623

In [6]:
import timeit

def function1():
    gak(X, X, sigma=sigma)

def function2():
    ksiggak(X_np, X_np)

def function3():
    mine(X, X)

# Measure the execution time of function 1
execution_time1 = timeit.timeit(function1, number=1)

# Measure the execution time of function 2
execution_time2 = timeit.timeit(function2, number=10)

# Measure the execution time of function 3
execution_time3 = timeit.timeit(function3, number=10)

print("Execution time of function 1:", execution_time1)
print("Execution time of function 2:", execution_time2)
print("Execution time of function 3:", execution_time3)


Execution time of function 2: 0.09977894100029516
Execution time of function 3: 1.3846606770002836


In [ ]:
#### test dimensions of TimeSeriesKernels ####

N=3
N2=4
T, d = 40, 5
X = torch.randn(N, T, d, dtype=torch.float64) / d
Y = torch.randn(N2, T, d, dtype=torch.float64) / d

inputs = [
    (X, X),
    (X, Y),
    (X[0], X[0]),
    (X[0], Y[0]),
    (X[0], Y),
    (X, Y[0]),
]
diag_inputs = [
    (X, X),
    (X, Y[:N]),
    (X[0], X[0]),
    (X[0], Y[0]),
]
def test_kernel(ker: TimeSeriesKernel, inputs, diag=False):
    print(ker)
    for X, Y in inputs:
        out = ker(X, Y, diag, normalize=False)
        out_normalize = ker(X, Y, diag, normalize=True)
        print(out)
        print(out_normalize)
        print(out.shape)
    print()


sigker = TruncSigKernel(static_kernel=RBFKernel(), 
                        trunc_level=4, 
                        geo_order=1,
                        only_last=True,)
test_kernel(sigker, inputs)
test_kernel(sigker, diag_inputs, True)

sigpde = SigPDEKernel(static_kernel=RBFKernel(),
                     dyadic_order=3,)
test_kernel(sigpde, inputs)
test_kernel(sigpde, diag_inputs, True)

intker = StaticIntegralKernel(static_kernel=RBFKernel())
test_kernel(intker, inputs)
test_kernel(intker, diag_inputs, True)

In [ ]:
import numpy as np
import ksig

# Number of signature levels to use.
n_levels = 5 

# Use the RBF kernel for vector-valued data as static (base) kernel.
static_kernel = ksig.static.kernels.RBFKernel() 

# Instantiate the signature kernel, which takes as input the static kernel.
n_levels = 5
order = 1
sig_kernel = ksig.kernels.SignatureKernel(n_levels=n_levels, order=order, static_kernel=static_kernel)

# Generate 10 sequences of length 50 with 5 channels.
n_seq, l_seq, n_feat = 10, 50, 5 
X = np.random.randn(n_seq, l_seq, n_feat)

# Sequence kernels take as input an array of sequences of ndim == 3,
# and work as a callable for computing the kernel matrix. 
K_XX = sig_kernel(X)  # K_XX has shape (10, 10).

# The diagonal kernel entries can also be computed.
K_X = sig_kernel(X, diag=True)  # K_X has shape (10,).

# Generate another array of 8 sequences of length 20 and 5 features.
n_seq2, l_seq2 = 8, 20
Y = np.random.randn(n_seq2, l_seq2, n_feat)

# Compute the kernel matrix between arrays X and Y.
K_XY = sig_kernel(X, Y)  # K_XY has shape (10, 8)
K_XY